# Labo 7 : classification de textes (pour la désambiguïsation lexicale)
Nathan Gonzalez Montes & Guidoux Vincent

## Objectif et plan

L’objectif de ce laboratoire est d’utiliser des méthodes d’apprentissage supervisé pour classifier des occurrences du mot interest selon leur sens : c’est la même tâche, avec les mêmes données, que le labo 6. Pour classifier les occurrences, on considère leur contexte (mots voisins), et on applique l’approche bayésienne vue en cours : on entraîne un classifieur à six classes (les six sens de interest annotés de 1 à 6) sur une partie des données et on le teste sur la partie restante.

Dans ce laboratoire, on explore deux façons différentes de coder les traits (features) pour cette tâche. Dans les deux cas, on entraînera un NaiveBayesClassifier fourni par NLTK.1 Les deux façons sont :
1. Constituer un vocabulaire des mots qui apparaissent dans le voisinage de interest et définir ces mots comme traits. Pour chaque occurrence de interest, on extrait la valeur de ces traits sous la forme `{(‘rate’ : True), (‘in’ : False), … }` et on ajoute la classe (de 1 à 6).
2. Si word-1 est le mot précédant l’occurrence de interest, on définit comme traits word-n, …, word-2, word-1, word+1, word+2, …, word+n (une fenêtre de taille 2n autour de interest). Les valeurs possibles de ces traits sont cette fois-ci les mots observés, ou ‘NONE’ si la fenêtre dépasse les limites de la phrase. Pour chaque occurrence de interest, on extrait la valeur de ces traits sous la forme `{(‘word-1’ : ‘his’), (‘word+1’ : ‘in’), … }` et on ajoute la classe (de 1 à 6).

Dans les deux cas, il faut diviser les 2368 occurrences de interest en un jeu d’entraînement et un jeu de test, en respectant la proportion initiale de chaque sens. Puis on entraîne un classifieur bayésien naïf en respectant le format de données indiqué par NLTK2, et on teste la performance du classifieur entraîné. L’objectif est de trouver les paramètres qui conduisent aux meilleurs scores de WSD.

## Importation

In [1]:
import nltk.tokenize
import numpy as np
import re
import random
import time
import pylab as plt
import matplotlib
%matplotlib inline

## Liste des noms propres, ponctuation :

## Étapes proposées

### A. Traits lexicaux : présence ou absence de mots dans le voisinage de interest

#### 1. Le fichier de données se trouve à http://www.d.umn.edu/~tpederse/data.html – chercher « interest » vers la fin de la page, et prendre le fichier marqué comme « original format without POS tags » (le même qu’au labo 6). Lire le fichier et générer une liste de listes de mots (une liste par phrase) appelée `tokenized_sentences`.

In [2]:
ponctuations = [',', '.', '`', '\'','-', ';', ':', ')', '&', '{', '(', '}', '=']

In [3]:
filepath = 'data/interest-original.txt' 

try:  
    fp = open(filepath, 'r',encoding="utf-8")
    raw_sentences = fp.read()
finally:  
    fp.close()
    
for ponct in ponctuations:
    raw_sentences = raw_sentences.replace(ponct, ' ') # clear the data

raw_sentences = re.sub(' [0-9]+', '', raw_sentences) # clear the data

raw_sentences = raw_sentences.split('\n$$\n') # clear the data

raw_sentences = raw_sentences[:-1] # la dernière phrase est vide

tokenized_sentences = [nltk.word_tokenize(sent) for sent in raw_sentences] 

print("Il y a {} phrases ".format(len(tokenized_sentences)))

Il y a 2368 phrases 


#### 2. Définir une variable `window_size`, par exemple égale à 3 (on la fera varier plus tard), et une liste vide de mots `word_list`. Parcourir les `tokenized_sentences` et pour chaque phrase ajouter les mots voisins de interest (i.e. situés à une distance inférieure ou égale à `window_size`) dans la liste de mots `word_list`. Combien de mots contient celle-ci à la fin ? Tokens ou types ?

In [4]:
def windowation(tokenized_sentences, windows_size):
    
    word_dict = {1: [], 2: [], 3: [], 4: [], 5: [], 6: []}
    vocabulary = set()
    word_list = []
       
    for sentence in (tokenized_sentences):  
        i = None
        current_sens = None
        for def_i in range(1,7):
            current_sens = def_i
            if "interest_{}".format(def_i) in sentence:
                i = sentence.index("interest_{}".format(def_i))
                break
            elif "interests_{}".format(def_i) in sentence:
                i = sentence.index("interests_{}".format(def_i))
                break
        if i != None:
            for index in range(-windows_size, windows_size+1):
                current_index = index + i
                if current_index > 0 and current_index < len(sentence) and index != 0:
                    current_word = sentence[current_index]
                    word_dict[current_sens].append(current_word)
                    vocabulary.add(current_word)
                    word_list.append(current_word)
        else:
            print("ERROR")
            
    return (word_dict, vocabulary, word_list)

In [5]:
windows_size = 3

word_dict, vocabulary, word_list = windowation(tokenized_sentences, windows_size)

In [6]:
somme = 0

for i in range(1,7):
    current_list = word_dict[i]
    print("word_list {} : {}".format(i, len(current_list)))
    somme += len(current_list)

word_list 1 : 1954
word_list 2 : 65
word_list 3 : 309
word_list 4 : 935
word_list 5 : 2772
word_list 6 : 6759


In [7]:
print("nombre total d'occurances : {}".format(somme))
print("nombre de mots différents : {}".format(len(vocabulary)))

nombre total d'occurances : 12794
nombre de mots différents : 2274


#### 3. À l’aide d’un objet `NLTK` de type `FreqDist`, sélectionner parmi les mots de `word_list` les `N` plus fréquents, dans une nouvelle liste appelée `vocabulary` (p.ex. `N = 500`, mais on le fera varier). Affichez les 50 mots les plus fréquents. Est-ce une bonne idée d’enlever les stopwords ?

In [8]:
N = 500
vocabulary = nltk.FreqDist(word_list).most_common(N)
vocabulary = np.array(vocabulary)
vocabulary = vocabulary[:, 0] # Nous ne gardons que la première colonne

In [9]:
nltk.FreqDist(word_list).most_common(50)

[('in', 770),
 ('rates', 624),
 ('the', 608),
 ('and', 408),
 ('to', 379),
 ('of', 333),
 ('a', 292),
 ('s', 181),
 ('on', 172),
 ('%', 163),
 ('rate', 144),
 ('its', 134),
 ('payments', 112),
 ('that', 103),
 ('are', 102),
 ('for', 93),
 ('has', 91),
 ('with', 89),
 ('lower', 82),
 ('an', 82),
 ('short', 80),
 ('is', 72),
 ('have', 69),
 ('by', 68),
 ('at', 65),
 ('will', 64),
 ('high', 63),
 ('from', 63),
 ('u', 60),
 ('term', 52),
 ('$', 52),
 ('company', 49),
 ('foreign', 48),
 ('annual', 48),
 ('their', 48),
 ('which', 47),
 ('or', 46),
 ('minority', 46),
 ('bonds', 44),
 ('higher', 43),
 ('said', 43),
 ('income', 42),
 ('other', 41),
 ('as', 41),
 ('pay', 40),
 ('it', 39),
 ('below', 39),
 ('t', 36),
 ('be', 35),
 ('up', 35)]

**Est-ce une bonne idée d’enlever les stopwords ?**
> blabla

#### 4. Parcourir à nouveau les `tokenized_sentences` et pour chaque phrase créer un couple (dictionnaire, sens), où le dictionnaire regroupe les traits et leurs valeurs, et le sens est un nombre de 1 à 6 indiquant le sens de interest. Les couples pour toutes les phrases seront rassemblés dans une liste appelée `feature_sets`.
- Prendre modèle sur https://www.nltk.org/book/ch06.html (début du 1.2)
- Pour le dictionnaire, il faut créer un trait pour chaque mot de vocabulary, et examiner si ce mot est présent dans une fenêtre de taille window_size autour de l’occurrence de interest : si oui, le trait est True, sinon il est False. Par exemple, on aboutit à : {'contains(the)': False, 'contains(,)': True, 'contains(rates)': True, …}.
- Ajouter aussi le trait ‘word0’ qui note si l’occurrence est interest ou interests (pluriel).

In [10]:
def gender_features(sentence, windows_size, vocabulary):
    feature_set = {}
    current_sens = None
    word0 = False
    for def_i in range(1,7):
        current_sens = def_i
        current_interest = "interest_{}".format(def_i)
        current_interests = "interests_{}".format(def_i)
        if current_interest in sentence:
            i = sentence.index(current_interest)
            break
        elif current_interests in sentence:
            i = sentence.index(current_interests)
            word0 = True
            break
    feature_set['word0'] = word0
    if i != None:
        neighbours = []
        for index in range(-windows_size, windows_size+1):
            current_index = index + i
            if current_index > 0 and current_index < len(sentence) and index != 0:
                neighbours.append(sentence[current_index])
        for trait in vocabulary:
            if trait in neighbours:
                feature_set[trait] = True
            else:
                feature_set[trait] = False
            
    else:
        print("ERROR")
    return (feature_set,current_sens)

In [11]:
feature_sets = []

for sentence in tokenized_sentences:
    feature_sets.append(gender_features(sentence, windows_size, vocabulary))
len(feature_sets)

2368

##### 4.3 Combien d’occurrences pour chaque sens de interest y a-t-il dans feature_sets ?

In [12]:
occurrences = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}
somme = 0
for feature_set in feature_sets:
    occurrences[feature_set[1]] += 1

In [13]:
occurrences

{1: 361, 2: 11, 3: 66, 4: 178, 5: 500, 6: 1252}

#### 5. Diviser les données de feature_sets en deux sous-ensembles : l’un comportant 80% des données est le `train_set`, et l’autre (20%) est le `test_set`. Attention, il faut respecter deux conditions :
- Mélanger avec shuffle() les occurrences avant de prendre les 80% premières pour le `train_set`et les 20% restantes dans le test_set.
- Chaque sens doit être présent dans les mêmes proportions dans train_set et dans test_set (donc il faut faire la division de manière séparée pour chaque sens).

In [14]:
def separate_train_test(feature_sets):

    random.shuffle(feature_sets)

    occurence_in_train = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}
    train_set = []
    test_set = []
    for feature_set in feature_sets:
        current_sens = feature_set[1]
        if occurence_in_train[current_sens] < (occurrences[current_sens] * 0.8):
            train_set.append(feature_set)
            occurence_in_train[current_sens] += 1
        else:
            test_set.append(feature_set)
    return (train_set, test_set)

In [15]:
train_set, test_set = separate_train_test(feature_sets)

print("Il y a {} couples dans train et {} dans test".format(len(train_set), len(test_set)))

Il y a 1896 couples dans train et 472 dans test


#### 6. Entraîner un classifieur de type `NaiveBayesClassifier` de `NLTK` sur `train_set`

In [16]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

**puis le tester sur les données de `test_set`. Quelle est la précision (`accuracy`) atteinte ?**

In [17]:
print("La précision atteinte est de {:.2f}".format(nltk.classify.accuracy(classifier, test_set)))

La précision atteinte est de 0.88


#### 7. Adapter le code précédent pour effectuer plusieurs divisions des données en train et test (par exemple 10), et calculer la moyenne des scores obtenus. Comment se compare cette moyenne avec votre premier résultat ?

In [18]:
def train_test(feature_sets):
    train_set, test_set = separate_train_test(feature_sets)
    classifier = nltk.NaiveBayesClassifier.train(train_set)
    accuracy = nltk.classify.accuracy(classifier, test_set)
    return accuracy

In [ ]:
N_iteration = 10
accuracies = []
for iteration in range(N_iteration):
    print("-----itération n°{}-----".format(iteration))
    accuracies.append(train_test(feature_sets))

-----itération n°0-----
-----itération n°1-----
-----itération n°2-----
-----itération n°3-----
-----itération n°4-----
-----itération n°5-----
-----itération n°6-----
-----itération n°7-----
-----itération n°8-----
-----itération n°9-----


In [ ]:
print("la moyenne des scores obtenus est de {:.2f} ".format(np.mean(accuracies)))

la moyenne des scores obtenus est de 0.88 


#### 8. Cherchez les meilleurs paramètres pour la taille de la fenêtre (p.ex. 1, 3, 5, 7, 11) et la taille du vocabulaire (50, 100, 200, 500, 1000 mots). Combien d’expériences faut-il exécuter ? Quelle est la meilleure combinaison fenêtre x vocabulaire et quel est le score moyen obtenu ?

In [ ]:
N_iteration = 10
windows_sizes = [1,3,5,7,11]
# windows_sizes = [1,3]
vocabulary_sizes =[50, 100, 200, 500, 1000]
# vocabulary_sizes =[50,100]

In [ ]:
scores = np.empty((len(vocabulary_sizes),len(windows_sizes)),dtype=float)
np.shape(scores)

(5, 5)

In [ ]:
for i, windows_size in enumerate(windows_sizes):
    for j, vocabulary_size in enumerate(vocabulary_sizes):
        start = time.time()
        print("\n------------ windows_size : {}; vocabulary_size : {} ------------".format(windows_size, vocabulary_size))
        
        word_dict, vocabulary, word_list = windowation(tokenized_sentences, windows_size)
        
        somme = 0
        
        for sens_i in range(1,7):
            current_list = word_dict[sens_i]
#             print("word_list {} : {}".format(i, len(current_list)))
            somme += len(current_list)
        
#         print("nombre total d'occurances : {}".format(somme))
#         print("nombre de mots différents : {}".format(len(vocabulary)))
        
        vocabulary = nltk.FreqDist(word_list).most_common(N)
        vocabulary = np.array(vocabulary)
        vocabulary = vocabulary[:, 0]
        
        feature_sets = []

        for sentence in tokenized_sentences:
            feature_sets.append(gender_features(sentence, windows_size, vocabulary))
        len(feature_sets)
        
        occurrences = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}
        for feature_set in feature_sets:
            occurrences[feature_set[1]] += 1
        
        train_set, test_set = separate_train_test(feature_sets)
#         print("Il y a {} couples dans train et {} dans test".format(len(train_set), len(test_set)))
        
        accuracies = []
        for iteration in range(N_iteration):
            accuracies.append(train_test(feature_sets))
            
        current_accuracy = np.mean(accuracies)
        
        scores[i][j] = current_accuracy
        
        end = time. time()

        execution_time = end-start
        print("la moyenne des scores obtenus est de {:.3f} en {:.0f} secondes ".format(current_accuracy, execution_time))



------------ windows_size : 1; vocabulary_size : 50 ------------
la moyenne des scores obtenus est de 0.847 en 57 secondes 

------------ windows_size : 1; vocabulary_size : 100 ------------
la moyenne des scores obtenus est de 0.853 en 59 secondes 

------------ windows_size : 1; vocabulary_size : 200 ------------
la moyenne des scores obtenus est de 0.857 en 55 secondes 

------------ windows_size : 1; vocabulary_size : 500 ------------


In [ ]:
scores

In [ ]:
# plot
fig, ax = plt.subplots(num=None,figsize=(10,14),dpi=250) 
heatmap = ax.pcolor(scores, cmap=plt.cm.gray)

ax.set_xticks(np.arange(scores.shape[1])+0.5, minor=False)
ax.set_yticks(np.arange(scores.shape[0])+0.5, minor=False)

ax.set_frame_on(False)
ax.xaxis.tick_top()
ax.grid(False)
plt.xlim([0,np.shape(scores)[1]])

ax.set_yticklabels(windows_sizes, minor=False) 
ax.set_xticklabels(vocabulary_sizes, minor=False)

plt.xticks(rotation=90) # rotate xlabels
matplotlib.rcParams['xtick.labelsize'] = 10

cbar = fig.colorbar(heatmap, orientation='horizontal')
cbar.set_label('Scores')

### B. Traits lexicaux positionnels : valeurs des mots précédant/suivant interest
Pour cette deuxième partie, on réutilisera beaucoup d’éléments de la première. Seule la nature des
traits utilisés et leur extraction vont changer.

#### 1. Partir de la liste de listes de mots (une liste par phrase) précédente, appelée tokenized_sentences.

#### 2. Définir une variable window_size2, par exemple égale à 3 (on la fera varier plus tard).

#### 3. Parcourir les tokenized_sentences et pour chaque phrase créer un couple (dictionnaire, sens), où le dictionnaire regroupe les traits et leurs valeurs, et le sens est un nombre de 1 à 6 indiquant le sens de interest. Les couples pour toutes les phrases seront rassemblés dans une nouvelle liste appelée feature_sets2.

##### 3.1 Pour le dictionnaire de traits, il faut cette fois-ci créer un trait pour chaque position relative par rapport à interest, donc ‘word-1’, ‘word+1’, etc. (jusqu’à window_size2). La valeur du trait sera le mot trouvé à cette position, ou ‘NONE’ si on sort de la phrase. Par exemple {(‘word-1’ : ‘his’), (‘word+1’ : ‘in’), … }.

##### 3.2 Ajouter aussi le trait ‘word0’ qui note si l’occurrence est interest ou interests (pluriel).

#### 4. Diviser les données de feature_sets2 en deux sous-ensembles (80%/20%) appelés train_set2 et test_set2 avec la même procédure qu’à la partie A.

#### 5. Entraîner un classifieur de type NaiveBayesClassifier de NLTK sur train_set2, puis le tester sur les données de test_set2. Quelle est la précision (accuracy) atteinte ?

#### 6. Effectuer plusieurs divisions des données en train et test (par exemple 10), et calculer la moyenne des scores obtenus. Comment se compare cette moyenne avec votre premier résultat ?

#### 7. Cherchez les meilleurs paramètres pour la taille de la fenêtre (p.ex. entre 1 et 15). Quelle est la meilleure valeur et quel est le score moyen obtenu ?

#### 8. Quelle est le meilleur score obtenu entre (A) et (B) ?

#### 9. Thème de réflexion facultatif : les différences des scores sont-elles statistiquement significatives ?

Merci d’envoyer votre notebook Jupyter par email au professeur avant le **lundi 27 mai à 23h59**.